In [81]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import loginInfo
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time

In [61]:
def wait_and_click_element(driver, xpath, timeout=10):
    """
    Waits for an element specified by the given XPath to be present on the page,
    and then clicks the element.

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        xpath (str): The XPath locator for the element to be clicked.
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage:
    wait_and_click_element(driver, "/html/body/app-component/nav-component/div[2]/div[2]/button[10]", max_wait_time=100)
    """

    wait = WebDriverWait(driver, timeout)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    element.click()

def wait_and_type_element(driver, xpath, text, timeout=10):
    """
    Waits for an element specified by the given XPath to be present on the page,
    and then clicks the element.

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        xpath (str): The XPath locator for the element to be typed in.
        text (str): The string to be typed in the textbox.
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage:
    wait_and_click_element(driver, "/html/body/app-component/nav-component/div[2]/div[2]/button[10]", "This will be typed", max_wait_time=100)
    """

    wait = WebDriverWait(driver, timeout)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    element.send_keys(text)

In [79]:

def find_inventory_element_and_click(driver, source, timeout = 100):
    """
    Finds the element in the inventory page, and then clicks on the element

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        source (str): The source of the image to be located. e.g. ".//img[@src='/assets/items/gem-ruby.png']"
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage: 
    find_inventory_element_and_click(driver, source = ".//img[@src='/assets/items/gem-ruby.png']")
    
    """
    # Assuming the webpage is already loaded
    base_xpath = "/html/body/app-component/div/div/div/inventory-page/div/div[1]/div/div[3]/button["
    found = False

    for i in range(1, timeout + 1):
        # Construct the full XPath for each button
        full_xpath = base_xpath + str(i) + "]"

        try:
            # Find the button element
            element = driver.find_element(By.XPATH, full_xpath)

            # Check if the button contains the Ruby image
            image = element.find_element(By.XPATH, source)
            element.click()
        except NoSuchElementException:
            filler = 1+1
        else:
            found = True

        # Stop after max_items even if desired item not found
        if i == timeout or found == True:
            break

def find_inventory_elements(driver, max_retries=3, timeout=30):
    """
    Finds all inventory elements using multiple strategies and returns them in an array.

    Args:
    driver (WebDriver): The Selenium WebDriver instance.
    max_retries (int): Maximum number of retries for finding elements.
    timeout (int): Maximum wait time for elements to be present.

    Returns:
    list: A list of dictionaries, each containing 'name', 'xpath', and 'quantity' for an inventory item.
    """
    inventory_items = []
    strategies = [
        ("//button[contains(@class, 'item')]", ".//img", ".//*[contains(@class, 'amount') or contains(@class, 'quantity')]"),
        ("//div[contains(@class, 'item')]", ".//img", ".//*[contains(@class, 'amount') or contains(@class, 'quantity')]"),
        ("//*[contains(@class, 'inventory-item')]", ".//img", ".//*[contains(@class, 'amount') or contains(@class, 'quantity')]")
    ]

    for attempt in range(max_retries):
        for base_xpath, img_xpath, quantity_xpath in strategies:
            try:
                # Wait for the inventory items to be present
                WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((By.XPATH, base_xpath))
                )
                
                # Find all inventory items
                items = driver.find_elements(By.XPATH, base_xpath)

                for i, item in enumerate(items, 1):
                    try:
                        # Find the image element
                        img_element = item.find_element(By.XPATH, img_xpath)
                        
                        # Extract the item name from the image source
                        src = img_element.get_attribute("src")
                        item_name = src.split("/")[-1].split(".")[0]
                        
                        # Find the quantity element
                        quantity_element = item.find_element(By.XPATH, quantity_xpath)
                        quantity = quantity_element.text
                        
                        inventory_items.append({
                            "name": item_name,
                            "xpath": f"({base_xpath})[{i}]",
                            "quantity": quantity
                        })
                    
                    except (NoSuchElementException, StaleElementReferenceException) as e:
                        print(f"Error processing item {i}: {str(e)}")

                if inventory_items:
                    return inventory_items
            
            except TimeoutException:
                print(f"Timeout waiting for items with xpath: {base_xpath}")
            
            except Exception as e:
                print(f"An unexpected error occurred: {str(e)}")

    print("Failed")
    return inventory_items

In [87]:
def click_menu_item(driver, menu_text, timeout=10, max_retries=3):
    """
    Clicks on a menu item based on its text content.
    Args:
    driver (WebDriver): The Selenium WebDriver instance.
    menu_text (str): The exact text of the menu item to click.
    timeout (int): Maximum wait time for elements to be present.
    max_retries (int): Maximum number of retries for finding and clicking the menu item.
    """
    for attempt in range(max_retries):
        print(f"Attempt {attempt + 1} of {max_retries} to click '{menu_text}'")
        try:
            # Wait for any button to be present (adjust if there's a more specific container)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.TAG_NAME, "button"))
            )

            # Find all buttons
            buttons = driver.find_elements(By.TAG_NAME, "button")

            for button in buttons:
                # Check button text and text of any child elements
                button_text = button.text.strip().lower()
                if menu_text.lower() in button_text:
                    # If text matches, try to click the button
                    try:
                        button.click()
                        print(f"Successfully clicked on '{menu_text}'")
                        return True
                    except Exception as e:
                        print(f"Found button but couldn't click: {str(e)}")
                        continue

            print(f"Could not find clickable button with text '{menu_text}' on attempt {attempt + 1}")
        except Exception as e:
            print(f"An error occurred: {str(e)}")

        # Wait before retrying
        time.sleep(2)

    print(f"Failed to click on '{menu_text}' after {max_retries} attempts")
    return False

In [85]:
driver = webdriver.Chrome()
driver.get("https://ironwoodrpg.com")

# Sign into the website and go to the main page
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/div[4]/button[2]' , timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[4]/button' , timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[1]/input', loginInfo.usernamelist[0], timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[3]/input', loginInfo.passwordlist[0], timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/button' , timeout=100)


In [92]:
click_menu_item(driver, "Smelting")

Attempt 1 of 3 to click 'Smelting'
Successfully clicked on 'Smelting'


True

In [80]:
find_inventory_elements(driver)

Attempt 1 of 3
Found 7 potential inventory items using xpath: //button[contains(@class, 'item')]
Added item: wood-spruce, Quantity: 44.8K
Added item: wood-pine, Quantity: 103.8K
Added item: rock-copper, Quantity: 8.9K
Added item: gem-ruby, Quantity: 99
Added item: food-grapes, Quantity: 2.3K
Added item: food-apple, Quantity: 5.3K
Added item: challenge-scroll, Quantity: 19


[{'name': 'wood-spruce',
  'xpath': "(//button[contains(@class, 'item')])[1]",
  'quantity': '44.8K'},
 {'name': 'wood-pine',
  'xpath': "(//button[contains(@class, 'item')])[2]",
  'quantity': '103.8K'},
 {'name': 'rock-copper',
  'xpath': "(//button[contains(@class, 'item')])[3]",
  'quantity': '8.9K'},
 {'name': 'gem-ruby',
  'xpath': "(//button[contains(@class, 'item')])[4]",
  'quantity': '99'},
 {'name': 'food-grapes',
  'xpath': "(//button[contains(@class, 'item')])[5]",
  'quantity': '2.3K'},
 {'name': 'food-apple',
  'xpath': "(//button[contains(@class, 'item')])[6]",
  'quantity': '5.3K'},
 {'name': 'challenge-scroll',
  'xpath': "(//button[contains(@class, 'item')])[7]",
  'quantity': '19'}]

In [84]:
import subprocess

def kill_chrome():
    # run the 'pkill' command to kill all 'chrome' processes
    subprocess.run(['pkill', 'Google Chrome'])

kill_chrome()